# 지도학습 이해


---
---



## **1. 지도학습(supervised learning) 이해**
- 정답(label)이 주어진 데이터를 이용해 모델을 학습시키는 기계학습 방식
- 모델은 입력(X)과 출력(Y) 간의 관계를 학습하고, 새로운 입력에 대한 예측을 수행

### **[실습] 자연어 처리(NLP) 분야에서의 지도학습(분류) 예**


#### **예제1. 텍스트 분류(뉴스 주제 분류)**
- **태스크** : 뉴스 주제 분류
- **문제유형** :  
- **데이터셋** :  
- **특징추출** : TF-IDF(N-gram: bigram)
- **분류기** :  
- **핵심내용**: 가장 기본적인 문서 분류 파이프라인

In [ ]:
# --- 1. 텍스트 분류: 20 Newsgroups ---
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score


# 1) 데이터 로드 (훈련/테스트 분리 제공)
cats = ['sci.space', 'rec.sport.baseball', 'comp.graphics', 'talk.politics.mideast']
train = fetch_20newsgroups(subset='train', categories=cats, remove=('headers','footers','quotes'))
test  = fetch_20newsgroups(subset='test',  categories=cats, remove=('headers','footers','quotes'))


# 2) 파이프라인: TF-IDF → 로지스틱 회귀(멀티클래스)
pipe = make_pipeline(TfidfVectorizer(min_df=3, ngram_range=(1,2)),
                     LogisticRegression(max_iter=300))


# 3) 학습
pipe.fit(train.data, train.target)


# 4) 평가
pred = pipe.predict(test.data)
print("Accuracy:", accuracy_score(test.target, pred))
print(classification_report(test.target, pred, target_names=test.target_names))


# 5) 사용 예시
sample = ["SpaceX just launched another rocket to the ISS.",
          "OpenGL shaders can speed up graphics rendering.",
          "The team won the baseball world series!",
          "The negotiation in the middle east escalated."]
print(pipe.predict(sample))          # 예측된 정수 라벨
print([test.target_names[i] for i in pipe.predict(sample)])  # 라벨 이름


#### **예제2. 감성 분류(긍/부정)**
NLTK의 movie_reviews 코퍼스(긍/부정 라벨)를 사용해 문서 분류(이진 분류)를 수행합니다.

- **태스크** : 감석 분석
- **문제유형** :  
- **데이터셋** :  
- **특징추출** : TF-IDF(N-gram: bigram)
- **분류기** :  
- **핵심내용**: 리뷰 텍스트의 긍/부정 판

In [ ]:
# --- 2. 감성 분석: NLTK movie_reviews (긍/부정) ---
import nltk
nltk.download('movie_reviews')

from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score

# 1) 문서/라벨 구성
docs = []
labels = []
for cat in movie_reviews.categories():          # 'pos', 'neg'
    for fid in movie_reviews.fileids(cat):
        docs.append(movie_reviews.raw(fid))
        labels.append(cat)

# 2) 학습/검증 분리
X_train, X_test, y_train, y_test = train_test_split(
    docs, labels, test_size=0.2, random_state=42, stratify=labels
)

# 3) 파이프라인: TF-IDF → Linear SVM
pipe = make_pipeline(TfidfVectorizer(min_df=3, ngram_range=(1,2)),
                     LinearSVC())

# 4) 학습 & 평가
pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

# 5) 사용 예시
samples = [
    "What a wonderful movie! Brilliant acting and a touching story.",
    "It was boring and way too long. I wouldn't recommend it."
]
print(pipe.predict(samples))  # ['pos', 'neg'] 예상


#### **예제3. 품사 태깅(POS Tagging)**
토큰 단위로 품사 라벨(POS tag) 를 예측하는 시퀀스 라벨링을, 간단한 맥락 기반 특징 + 로지스틱 회귀로 구현
- **태스크** : 품사 태깅
- **문제유형** : 시퀀스 라벨링(토큰 단위 분류)
- **데이터셋** :  
- **특징추출** : 주변 단어/점미사 등 수작업 특징
- **분류기** :  
- **핵심내용**: 지도학습의 핵심인 특징-->라벨) 체

In [ ]:
# --- 3. 품사 태깅: NLTK treebank + 간단 특징 + 로지스틱 회귀 ---
import nltk
nltk.download('treebank')
nltk.download('universal_tagset')

from nltk.corpus import treebank
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

# 1) (단어, 태그) 시퀀스 불러오기 (보편 태그셋 사용: NOUN, VERB, ADJ 등)
sents = treebank.tagged_sents(tagset='universal')

# 간단한 특징 함수: 현재/이전/다음 단어, 접미사/대문자/숫자 등
def token_features(sent_words, i):
    w = sent_words[i]
    prev_w = sent_words[i-1] if i-1 >= 0 else "<BOS>"
    next_w = sent_words[i+1] if i+1 < len(sent_words) else "<EOS>"
    feats = {
        "w.lower": w.lower(),
        "w.isupper": w.isupper(),
        "w.isdigit": w.isdigit(),
        "w.suffix2": w[-2:].lower(),
        "w.suffix3": w[-3:].lower(),
        "prev.lower": prev_w.lower(),
        "next.lower": next_w.lower(),
        "prev.isupper": prev_w.isupper() if isinstance(prev_w, str) else False,
        "next.isupper": next_w.isupper() if isinstance(next_w, str) else False,
    }
    return feats

# 2) 특징/라벨 벡터 만들기
X_dict, y = [], []
for sent in sents:
    words = [w for w, t in sent]
    tags  = [t for w, t in sent]
    for i in range(len(words)):
        X_dict.append(token_features(words, i))
        y.append(tags[i])

# 3) 훈련/테스트 분리
X_train_dict, X_test_dict, y_train, y_test = train_test_split(
    X_dict, y, test_size=0.2, random_state=42, stratify=y
)

# 4) Dict → 벡터화 → 로지스틱 회귀
vec = DictVectorizer(sparse=True)
X_train = vec.fit_transform(X_train_dict)
X_test  = vec.transform(X_test_dict)

clf = LogisticRegression(max_iter=300, n_jobs=None, multi_class='auto')
clf.fit(X_train, y_train)

# 5) 평가
pred = clf.predict(X_test)
print("Token-level Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))


### **[실습] 자연어 처리(NLP) 분야에서의 지도학습(회귀) 예**

#### **예제1 : 영화 리뷰 감정 점수 예측 (1-5점 척도)**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import re
from nltk.corpus import stopwords
import nltk

# 샘플 데이터 생성
reviews = [
    "This movie was absolutely amazing! Perfect story and acting.",
    "Great film with excellent cinematography and soundtrack.",
    "It was okay, nothing special but watchable.",
    "Not my favorite, but had some good moments.",
    "Terrible movie, waste of time and money.",
    "Outstanding performance by all actors!",
    "Average movie with predictable plot."
]

ratings = [5.0, 4.5, 3.0, 2.5, 1.0, 4.8, 3.2]

# 텍스트 전처리 함수
def preprocess_text(text):
    # 소문자 변환
    text = text.lower()
    # 특수문자 제거
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

# 전처리 적용
processed_reviews = [preprocess_text(review) for review in reviews]

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=100, stop_words='english')
X = vectorizer.fit_transform(processed_reviews)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, ratings, test_size=0.3, random_state=42
)

# 선형 회귀 모델 훈련
model = LinearRegression()
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse:.3f}")
print(f"R² Score: {r2:.3f}")
print(f"실제값: {y_test}")
print(f"예측값: {y_pred}")

print()
상대오차율 = ((y_test - y_pred) / y_test) * 100
print(f"상대오차율: {상대오차율}%")

#### **예제2 : 텍스트 가독성 점수 예측**

In [ ]:
!pip install textstat

In [ ]:
import textstat
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# 샘플 텍스트와 가독성 점수 (Flesch Reading Ease 기준)
texts = [
    "The cat sat on the mat.",
    "Artificial intelligence represents a paradigm shift in computational methodologies.",
    "Machine learning algorithms can process large amounts of data efficiently.",
    "Deep learning uses neural networks with multiple layers.",
    "This is a simple sentence that anyone can understand easily."
]

# 실제 Flesch Reading Ease 점수 계산
readability_scores = [textstat.flesch_reading_ease(text) for text in texts]

# 특징 추출 (문장 길이, 단어 수, 음절 수 등)
def extract_features(text):
    words = text.split()
    sentences = text.split('.')

    return [
        len(words),  # 단어 수
        len(sentences),  # 문장 수
        sum(len(word) for word in words) / len(words),  # 평균 단어 길이
        textstat.syllable_count(text),  # 음절 수
        len(text)  # 총 글자 수
    ]

# 특징 벡터 생성
features = [extract_features(text) for text in texts]
X = np.array(features)
y = np.array(readability_scores)

# 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Random Forest 회귀 모델
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_scaled, y)

# 새로운 텍스트 예측
new_text = "Python is a programming language that is easy to learn and use."
new_features = scaler.transform([extract_features(new_text)])
predicted_score = rf_model.predict(new_features)[0]

print(f"새로운 텍스트: {new_text}")
print(f"예측된 가독성 점수: {predicted_score:.2f}")
print(f"실제 가독성 점수: {textstat.flesch_reading_ease(new_text):.2f}")

print()
actual = textstat.flesch_reading_ease(new_text)
상대오차율 = ((actual - predicted_score) / actual) * 100
print(f"상대오차율: {상대오차율:.2f}%")

#### **예제3 : 문서 유사도 점수 예측**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import SVR
import numpy as np

# 샘플 문서 쌍과 유사도 점수
doc_pairs = [
    ("The weather is nice today", "Today has beautiful weather"),
    ("I love programming", "Programming is my passion"),
    ("The cat is sleeping", "The dog is running"),
    ("Machine learning is fascinating", "AI and ML are interesting topics"),
    ("Python is great for data science", "Data analysis with Python is powerful")
]

# 실제 유사도 점수 (코사인 유사도 기반)
similarity_scores = []
for doc1, doc2 in doc_pairs:
    vectorizer = CountVectorizer().fit([doc1, doc2])
    vectors = vectorizer.transform([doc1, doc2])
    similarity = cosine_similarity(vectors[0], vectors[1])[0][0]
    similarity_scores.append(similarity)

# 특징 추출 (두 문서의 특징 결합)
def extract_pair_features(doc1, doc2):
    # 기본 통계
    len_diff = abs(len(doc1.split()) - len(doc2.split()))

    # 공통 단어 수
    words1 = set(doc1.lower().split())
    words2 = set(doc2.lower().split())
    common_words = len(words1.intersection(words2))

    # Jaccard 유사도
    jaccard_sim = len(words1.intersection(words2)) / len(words1.union(words2))

    return [len_diff, common_words, jaccard_sim]

# 특징 벡터 생성
features = [extract_pair_features(doc1, doc2) for doc1, doc2 in doc_pairs]
X = np.array(features)
y = np.array(similarity_scores)

# SVR 모델 훈련
svr_model = SVR(kernel='rbf', C=1.0, gamma='scale')
svr_model.fit(X, y)

# 예측 테스트
test_pair = ("Deep learning networks", "Neural networks for AI")
test_features = np.array([extract_pair_features(test_pair[0], test_pair[1])])
predicted_similarity = svr_model.predict(test_features)[0]

print(f"테스트 문서 쌍:")
print(f"문서 1: {test_pair[0]}")
print(f"문서 2: {test_pair[1]}")
print(f"예측된 유사도: {predicted_similarity:.3f}")

# 실제 코사인 유사도와 비교
vectorizer = CountVectorizer().fit(test_pair)
vectors = vectorizer.transform(test_pair)
actual_similarity = cosine_similarity(vectors[0], vectors[1])[0][0]
print(f"실제 코사인 유사도: {actual_similarity:.3f}")

print()
상대오차율 = ((actual_similarity - predicted_similarity) / actual_similarity) * 100
print(f"상대오차율: {상대오차율:.2f}%")



---



## **2. AI 모델링 프로세스**



---



## **3. 학습데이터 분할 방법**

### 예제 : K-Fold 교차검증 (회귀)

In [ ]:
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# 데이터 생성
# X, y = make_regression(n_samples=1000, n_features=10, noise=10, random_state=42)
# X, y = make_regression(
#     n_samples=200,      # 작은 데이터셋
#     n_features=15,      # 특징 수 증가
#     n_informative=8,    # 유용한 특징 수
#     noise=30,           # 높은 노이즈
#     random_state=42
# )
# # 더 큰 차이를 보이는 데이터
X, y = make_regression(
    n_samples=100,      # 매우 작은 데이터셋
    n_features=20,      # 특징 수 더 증가
    n_informative=5,    # 유용한 특징은 적게
    noise=50,           # 매우 높은 노이즈
    random_state=123    # 다른 시드값
)

def compare_evaluation_methods(X, y):
    """
    단일 분할 vs K-Fold 교차검증 간단 비교
    """
    print("=== 교차검증 유무 비교 ===")

    # 1. 단일 분할 (교차검증 X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    single_r2 = r2_score(y_test, y_pred)

    # 2. K-Fold 교차검증 (교차검증 O)
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(LinearRegression(), X, y, cv=kfold, scoring='r2')
    kfold_r2_mean = cv_scores.mean()
    kfold_r2_std = cv_scores.std()

    # 결과 출력
    print(f"단일 분할 R²: {single_r2:.3f}")
    print(f"K-Fold    R²: {kfold_r2_mean:.3f} ± {kfold_r2_std:.3f}")
    print(f"차이: {abs(single_r2 - kfold_r2_mean):.3f}")

    if kfold_r2_std < 0.05:
        print("✅ K-Fold 결과가 안정적입니다.")
    else:
        print("⚠️ 성능 변동이 있습니다.")

# 실행
compare_evaluation_methods(X, y)

### [실습] 학습 데이터 분할 방법 테스트
- 앞의 **2.감성 분류 과정**에서 학습/테스트 데이터 분류를 5:5, 9:1로 했을 때 모델의 정확도가 변동이 있는지 확인해 보세요.
- **8:1** = Accuracy: **0.87**
- **5:5** = Accuracy: **?**
- **9:1** = Accuracy: **?**

In [ ]:
# --- 2. 감성 분석: NLTK movie_reviews (긍/부정) ---
import nltk
nltk.download('movie_reviews')

from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score

# 1) 문서/라벨 구성
docs = []
labels = []
for cat in movie_reviews.categories():          # 'pos', 'neg'
    for fid in movie_reviews.fileids(cat):
        docs.append(movie_reviews.raw(fid))
        labels.append(cat)


- **조건1: 학습:테스트=5:5**

- **조건2: 학습:테스트=9:1**



---



## **4.모델 평가 방법**

### 예제 : 회귀모델 평가지표

|지표 | 공식 | 특징  | 해석 |
|---|---|---|--|
|MSE  | 평균((실제-예측)²) | 이상치에 민감 | 작을수록 좋음 |
|MAE평균 | (|실제-예측|) | 이상치에 강건 | 작을수록 좋음|
|R² | 1 - (오차분산/전체분산) | 설명력 측정 | 1에 가까울수록 좋음 |

In [ ]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 샘플 데이터 생성
X, y = make_regression(n_samples=100, n_features=5, noise=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 훈련 및 예측
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("=== 회귀 모델 평가지표 3가지 ===")
print(f"실제값 예시: {y_test[:5]}")
print(f"예측값 예시: {y_pred[:5]}")

# 1. MSE (Mean Squared Error) - 평균 제곱 오차
mse = mean_squared_error(y_test, y_pred)
print(f"\n1️⃣ MSE: {mse:.2f}")
print("   의미: 오차의 제곱 평균 (이상치에 민감)")
print("   해석: 값이 작을수록 좋음")

# 2. MAE (Mean Absolute Error) - 평균 절대 오차
mae = mean_absolute_error(y_test, y_pred)
print(f"\n2️⃣ MAE: {mae:.2f}")
print("   의미: 오차의 절댓값 평균 (이상치에 강건)")
print("   해석: 값이 작을수록 좋음, 직관적 이해 쉬움")

# 3. R² Score - 결정계수
r2 = r2_score(y_test, y_pred)
print(f"\n3️⃣ R² Score: {r2:.3f}")
print("   의미: 모델이 설명하는 분산 비율")
print("   해석: 0~1, 1에 가까울수록 좋음")

# 간단한 성능 평가
if r2 > 0.8:
    print("\n✅ 우수한 모델 성능")
elif r2 > 0.6:
    print("\n✅ 양호한 모델 성능")
else:
    print("\n⚠️ 모델 개선 필요")

### 예제 : 분류모델 평가지표

|지표 | 공식 | 언제 중요한가 | 해석|
|---|---|---|--|
|Accuracy | (TP+TN)/(TP+TN+FP+FN) | 클래스가 균형잡힌 경우 | 1에 가까울수록 좋음|
|Precision | TP/(TP+FP) | 오탐(False Positive)이 치명적일 때 | 1에 가까울수록 좋음|
|Recall | TP/(TP+FN) | 놓침(False Negative)이 치명적일 때 | 1에 가까울수록 좋음|
|F1-Score | 2×(Precision×Recall)/(Precision+Recall) | 균형잡힌 평가가 필요할 때 | 1에 가까울수록 좋음|

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# 샘플 분류 데이터 생성
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 훈련 및 예측
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("=== 분류 모델 평가지표 4가지 ===")
print(f"실제값 예시: {y_test[:10]}")
print(f"예측값 예시: {y_pred[:10]}")

# 1. Accuracy (정확도)
accuracy = accuracy_score(y_test, y_pred)
print(f"\n1️⃣ Accuracy: {accuracy:.3f}")
print("   의미: 전체 중 맞게 예측한 비율")
print("   해석: 1에 가까울수록 좋음")

# 2. Precision (정밀도)
precision = precision_score(y_test, y_pred)
print(f"\n2️⃣ Precision: {precision:.3f}")
print("   의미: 양성으로 예측한 것 중 실제 양성인 비율")
print("   해석: 거짓 양성(오탐)을 줄이고 싶을 때 중요")

# 3. Recall (재현율)
recall = recall_score(y_test, y_pred)
print(f"\n3️⃣ Recall: {recall:.3f}")
print("   의미: 실제 양성 중 올바르게 예측한 비율")
print("   해석: 거짓 음성(놓침)을 줄이고 싶을 때 중요")

# 4. F1-Score (F1 점수)
f1 = f1_score(y_test, y_pred)
print(f"\n4️⃣ F1-Score: {f1:.3f}")
print("   의미: Precision과 Recall의 조화평균")
print("   해석: 균형잡힌 성능 평가")

# 혼동행렬 (Confusion Matrix)
cm = confusion_matrix(y_test, y_pred)
print(f"\n📊 혼동행렬:")
print(f"   실제\\예측   0    1")
print(f"        0    {cm[0,0]:3}  {cm[0,1]:3}")
print(f"        1    {cm[1,0]:3}  {cm[1,1]:3}")

# 간단한 성능 평가
print(f"\n=== 성능 평가 ===")
if accuracy > 0.9:
    print("✅ 매우 우수한 성능")
elif accuracy > 0.8:
    print("✅ 우수한 성능")
elif accuracy > 0.7:
    print("✅ 양호한 성능")
else:
    print("⚠️ 성능 개선 필요")

# 정밀도 vs 재현율 트레이드오프
if precision > recall:
    print("📈 정밀도가 높음: 오탐이 적음")
else:
    print("📈 재현율이 높음: 놓치는 것이 적음")

### **[Tip] 모델의 파라미터 확인하는 방법**

In [ ]:
# 모델의 파라미터 확인하는 방법
print(model.get_params().keys())  # 파라미터 이름들
print(model.get_params())         # 파라미터 전체와 값



---

